In [ ]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time

from skimage.feature import SIFT
from sklearn.cluster import KMeans

In [ ]:
# Load images
imgs = []
data_path = 'cat_vs_dog'
for img_name in sorted(os.listdir(data_path)):
    img_path = os.path.join(data_path, img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imgs.append(img)

In [ ]:
# Extract SIFT features for all images
features_raw = []
feat_det = SIFT()
for img in imgs:
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    feat_det.detect_and_extract(img)
    feat = feat_det.descriptors
    feat = feat.astype(np.float32)
    features_raw.append(feat)
    print(feat.shape)

### Image retrieval with raw features (very slow query)

In [ ]:
def raw_feature_compare(feat1, feat2):
    """
    :param feat1: (N1, C)
    :param feat2: (N2, C)
    :return:
        dist: scalar
    """
    dist = np.expand_dims(feat1, axis=1) - np.expand_dims(feat2, axis=0)
    dist = np.linalg.norm(dist, axis=2)
    dist = dist.min(axis=1).mean()
    return dist

In [ ]:
# Give the query image
query_idx = 0
query_img = imgs[query_idx]
query_feat = features_raw[query_idx]
plt.imshow(query_img)
plt.show()

In [ ]:
n_top = 5
dists = []
start_time = time.time()

for img_idx, feat in enumerate(features_raw):
    dist = raw_feature_compare(query_feat, feat)
    dists.append(dist)
candidates = np.argsort(dists)[:n_top]

end_time = time.time()
print('time cost of image retrieval with raw features:', end_time - start_time)

In [ ]:
# Show image retrieval results
_, axes = plt.subplots(1, len(candidates))
for idx in range(len(candidates)):
    axes[idx].imshow(imgs[candidates[idx]])
plt.show()

### Image retrieval with Bag-of-Words features (slow preparation but very fast query)

In [ ]:
start_time = time.time()

# Construct the dictionary of visual words
features_all = np.concatenate(features_raw, 0)
num_words = 100
kmeans = KMeans(n_clusters=num_words, n_init=10)
kmeans.fit(features_all)
words = kmeans.cluster_centers_

# Convert raw features to Bag-of-Words features
def feat_to_bow(feat, words):
    """
    :param feat: (N1, C)
    :param words: (V, C)
    :return:
        feat_bow: (V,)
    """
    feat_bow = np.zeros([words.shape[0]], dtype=np.float32)
    for n in range(feat.shape[0]):
        dist = feat[n] - words
        dist = np.linalg.norm(dist, axis=1)
        word_idx = dist.argmin()
        feat_bow[word_idx] += 1
    return feat_bow

features_bow = []
for feat in features_raw:
    feat_bow = feat_to_bow(feat, words)
    features_bow.append(feat_bow)
    
end_time = time.time()
print('time cost of building BoW:', end_time - start_time)

In [ ]:
# Give the query image
query_idx = 0
query_img = imgs[query_idx]
query_feat = features_bow[query_idx]
plt.imshow(query_img)
plt.show()

In [ ]:
n_top = 5
dists = []
start_time = time.time()

for img_idx, feat in enumerate(features_bow):
    dist = query_feat - feat
    dist = np.linalg.norm(dist, axis=0)
    dists.append(dist)
candidates = np.argsort(dists)[:n_top]

end_time = time.time()
print('time cost with Bag-of-Words features:', end_time - start_time)

In [ ]:
# Show image retrieval results
_, axes = plt.subplots(1, len(candidates))
for idx in range(len(candidates)):
    axes[idx].imshow(imgs[candidates[idx]])
plt.show()